In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import json
import re
import pandas as pd
import numpy as np
import requests
from pyspark.sql.functions import lower
from pyspark.sql.functions import col


# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Domain EDA")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.port", "6066")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/07 07:24:13 WARN Utils: Your hostname, DESKTOP-VD1EQUL resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/07 07:24:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/07 07:24:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/07 07:24:14 WARN Utils: Service 'sparkDriver' could not bind on port 6066. Attempting port 6067.
24/10/07 07:24:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
domain_open_route = spark.read.parquet("../../data/raw/domain_listing_openRouteServices")
school_data = spark.read.csv("../../data/raw/schools.csv", header=True)
domain_cost_per_week = spark.read.parquet("../../data/raw/domain_cost_per_week")

In [3]:
VIC_amenities = spark.read.parquet("../../data/raw/VIC_amenities")

**Domain with Open Route Services**

In [4]:
domain_open_route.limit(1)

24/10/07 07:24:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving
2,1,1,https://www.domai...,"1/4 Hayley court,...",$460,Townhouse,-37.664825,145.040977,Mill Park VIC 3082,3082,460,The Austin School,38.22,38.22,Mercy Hospital fo...,22.44,34.99,Northend Medical ...,22.85,32.0,Westpac,24.19,24.19,Amalfi Pizza & Pasta,4.06,16.48,Broadmeadows Rail...,3.27,37.47


In [5]:
domain_open_route.count()

13328

In [6]:

# Step 1: Find duplicate keys
duplicates_df = domain_open_route.groupBy("url", "name") \
    .agg(F.count("*").alias("count")) \
    .filter(F.col("count") > 1)

# Step 2: Filter the original DataFrame to get all rows with duplicate keys
duplicate_keys = duplicates_df.select("url", "name")

# Join to get full rows of duplicates
duplicate_records_df = domain_open_route.join(duplicate_keys, on=["url", "name"], how="inner")

# Show all duplicate records
duplicate_records_df.limit(3)

url,name,bedrooms,bathrooms,car_parks,cost_text,type,latitude,longitude,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving
https://www.domai...,101/50 Victoria S...,2,1,1,$600 weekly,Apartment / Unit ...,-37.7690778,144.9792236,Brunswick East V...,3057,600,Sydney Road Commu...,13.43,13.43,Unknown,17.6,17.6,East Brunswick Me...,13.15,18.73,Commonwealth Bank,8.59,17.85,Rumi,16.26,16.26,North Melbourne R...,6.85,6.85
https://www.domai...,101/50 Victoria S...,2,1,1,$600 weekly,Apartment / Unit ...,-37.7690778,144.9792236,Brunswick East V...,3057,600,Sydney Road Commu...,13.43,13.43,Unknown,17.6,17.6,East Brunswick Me...,13.15,18.73,Commonwealth Bank,8.59,17.85,Rumi,16.26,16.26,North Melbourne R...,6.85,6.85
https://www.domai...,3/25 Lansdowne Ro...,3,1,1,$675,Apartment / Unit ...,-37.86387000000001,145.0065834,St Kilda East VI...,3183,675,Holmes Grammar Sc...,12.57,17.05,Orthodontist,19.52,19.52,Chapel Gate Medic...,2.26,12.66,Commonwealth Bank,5.12,13.28,Ecoblu,4.25,13.4,Caulfield Railway...,15.97,29.84


In [7]:
domain_open_route = domain_open_route.dropDuplicates()

In [8]:
domain_open_route.count()

9308

**Domain with Cost Per Week Modification**

In [9]:
domain_cost_per_week.limit(1)

bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week
3,2,0,https://www.domai...,"38 Airlie Avenue,...",$895 per week,House,-37.8563278,145.0077722,Prahran VIC 3181,3181,895,Sophia Mundi Stei...,13.19,144.07,Orthodontist,19.52,19.52,Chapel Gate Medic...,2.26,12.66,Commonwealth Bank,4.71,13.81,La Poele - modern...,1.28,10.12,Caulfield Railway...,15.97,29.84,weekly,895.0


In [10]:
domain_cost_per_week.count()

13328

In [11]:
# Step 1: Find duplicate keys
duplicates_df = domain_cost_per_week.groupBy("url", "name") \
    .agg(F.count("*").alias("count")) \
    .filter(F.col("count") > 1)

# Step 2: Filter the original DataFrame to get all rows with duplicate keys
duplicate_keys = duplicates_df.select("url", "name")

# Join to get full rows of duplicates
duplicate_records_df = domain_cost_per_week.join(duplicate_keys, on=["url", "name"], how="inner")

# Show all duplicate records
duplicate_records_df.limit(3)

url,name,bedrooms,bathrooms,car_parks,cost_text,type,latitude,longitude,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week
https://www.domai...,2/43 McNulty Driv...,3,2,2,$550,House,-38.2068047,146.5115895,Traralgon VIC 3844,3844,550,Kurnai College Un...,27.97,27.97,Heyfield Hospital,50.98,50.98,Breed Street Clinic,22.68,22.85,Bank of Australia,19.35,19.35,Unknown,40.9,40.9,Traralgon Railway...,15.47,20.03,weekly,550.0
https://www.domai...,2/43 McNulty Driv...,3,2,2,$550,House,-38.2068047,146.5115895,Traralgon VIC 3844,3844,550,Kurnai College Un...,27.97,27.97,Heyfield Hospital,50.98,50.98,Breed Street Clinic,22.68,22.85,Bank of Australia,19.35,19.35,Unknown,40.9,40.9,Traralgon Railway...,15.47,20.03,weekly,550.0
https://www.domai...,2/43 McNulty Driv...,3,2,2,$550,House,-38.2068047,146.5115895,Traralgon VIC 3844,3844,550,Kurnai College Un...,27.97,27.97,Heyfield Hospital,50.98,50.98,Breed Street Clinic,22.68,22.85,Bank of Australia,19.35,19.35,Unknown,40.9,40.9,Traralgon Railway...,15.47,20.03,weekly,550.0


In [12]:
domain_cost_per_week = domain_cost_per_week.dropDuplicates()

In [13]:
domain_cost_per_week.count()

9308

**To deal with messed up rows due to pyspark saving**

In [14]:
domain_cost_per_week = domain_cost_per_week.drop("car_parks", "cost_text",
                                                 "suburb", "postcode", "rent_pw",
                                                 "school_name", "school_walking",
                                                 "school_driving", "hospital_name",
                                                 "hospital_walking", "hospital_driving",
                                                 "doctors_name", "doctors_walking", 
                                                 "doctors_driving", "bank_name",
                                                 "bank_walking", "bank_driving",
                                                 "restaurant_name", "restaurant_walking",
                                                 "restaurant_driving", "PTV_name",
                                                 "PTV_walking", "PTV_driving")

In [15]:
domain_joined = domain_open_route.join(domain_cost_per_week, 
                                       on=["url", "name", 
                                           "latitude", "longitude",
                                           "bedrooms", "bathrooms",
                                           "type"], 
                                       how="left")  # Change "inner" to "left", "right", "outer" as needed

# Show the result
domain_joined.limit(1)

url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week
https://www.domai...,5/254-256 Dandeno...,-37.8593356,145.0009763,1,1,Apartment / Unit ...,1,$410 per week,St Kilda East VI...,3183,410,Holmes Grammar Sc...,12.57,17.05,Orthodontist,19.52,19.52,Chapel Gate Medic...,2.26,12.66,Commonwealth Bank,5.12,13.28,La Poele - modern...,1.28,10.12,Richmond Railway ...,1.64,45.69,weekly,410.0


In [16]:
domain_joined.count()

9308

In [17]:
domain_joined = domain_joined.filter(
    (F.col("postcode").isNotNull()) & 
    (F.trim(F.col("postcode")) != "")
)
domain_joined.count()

9256

In [18]:
# domain_cleaned = domain_joined.drop("url", "name", "latitude",
#                                     "longitude", "cost_text", 
#                                     "rent_pw", "school_name",
#                                     "hospital_name", "doctors_name",
#                                     "bank_name", "restaurant_name",
#                                     "PTV_name")

domain_cleaned = domain_joined

In [19]:
domain_cleaned.limit(1)

url,name,latitude,longitude,bedrooms,bathrooms,type,car_parks,cost_text,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving,rate_type,cost_per_week
https://www.domai...,5/254-256 Dandeno...,-37.8593356,145.0009763,1,1,Apartment / Unit ...,1,$410 per week,St Kilda East VI...,3183,410,Holmes Grammar Sc...,12.57,17.05,Orthodontist,19.52,19.52,Chapel Gate Medic...,2.26,12.66,Commonwealth Bank,5.12,13.28,La Poele - modern...,1.28,10.12,Richmond Railway ...,1.64,45.69,weekly,410.0


In [20]:
# Calculate 1st and 99th percentiles
percentiles = domain_cleaned.selectExpr("percentile_approx(cost_per_week, 0.01) as p1",
                                           "percentile_approx(cost_per_week, 0.99) as p99").first()

p1 = percentiles.p1
p99 = percentiles.p99

print(f"1st Percentile: {p1}, 99th Percentile: {p99}")

# Filter out values outside of the 1st and 99th percentiles
filtered_df = domain_cleaned.filter((col('cost_per_week') > p1) & (col('cost_per_week') < p99))

1st Percentile: 210.0, 99th Percentile: 2687.0


In [21]:
domain_cleaned_pd = filtered_df.toPandas()

In [22]:
domain_cleaned_pd.describe()

,bedrooms,bathrooms,car_parks,rent_pw,school_walking,school_driving,hospital_walking,hospital_driving,doctors_walking,doctors_driving,bank_walking,bank_driving,restaurant_walking,restaurant_driving,PTV_walking,PTV_driving,cost_per_week
count,9068.000000,9068.000000,9068.000000,9068.000000,9068.000000,9068.000000,9041.000000,9041.000000,9068.000000,9068.000000,9068.000000,9068.000000,9068.000000,9068.000000,9068.000000,9068.000000,9068.000000
mean,2.685708,1.574327,1.377702,632.822894,23.998999,41.023180,19.768172,22.863088,11.564373,18.517235,9.864198,18.659133,10.715957,17.210533,8.781230,38.341544,649.562155
std,1.083493,0.630543,1.009351,741.584404,27.623619,46.246862,11.215324,11.624789,7.571277,16.362329,8.219827,22.572978,10.263701,10.447934,7.914618,22.215953,321.404215
min,0.000000,0.000000,0.000000,1.000000,5.290000,9.300000,1.410000,1.710000,0.100000,0.100000,0.020000,0.460000,0.100000,0.130000,0.070000,0.110000,220.000000
25%,2.000000,1.000000,1.000000,480.000000,9.610000,17.050000,10.780000,11.000000,6.530000,12.180000,5.070000,12.380000,4.970000,11.560000,2.160000,25.360000,485.000000
50%,3.000000,2.000000,1.000000,560.000000,13.430000,32.200000,17.600000,20.320000,11.210000,15.770000,7.550000,14.150000,7.995000,15.010000,6.420000,32.420000,570.000000
75%,3.000000,2.000000,2.000000,680.000000,32.840000,38.220000,28.420000,34.400000,15.100000,20.160000,12.290000,18.280000,13.790000,19.760000,13.110000,45.690000,700.000000
max,9.000000,12.000000,19.000000,52000.000000,196.350000,196.350000,55.440000,55.440000,49.870000,156.370000,82.800000,225.120000,270.750000,161.750000,70.310000,99.610000,2672.000000


In [23]:
domain_cleaned_pd.isnull().sum()

url                    0
name                   0
latitude               0
longitude              0
bedrooms               0
bathrooms              0
type                   0
car_parks              0
cost_text              0
suburb                 0
postcode               0
rent_pw                0
school_name            0
school_walking         0
school_driving         0
hospital_name          0
hospital_walking      27
hospital_driving      27
doctors_name           0
doctors_walking        0
doctors_driving        0
bank_name              0
bank_walking           0
bank_driving           0
restaurant_name        0
restaurant_walking     0
restaurant_driving     0
PTV_name               0
PTV_walking            0
PTV_driving            0
rate_type              0
cost_per_week          0
dtype: int64

In [24]:
domain_cleaned_pd = domain_cleaned_pd.dropna()

In [29]:
domain_cleaned_spark = spark.createDataFrame(domain_cleaned_pd)

In [30]:
domain_cleaned_spark.write.mode('overwrite').parquet('../../data/curated/domain_cleaned_pyspark')

24/10/07 07:37:22 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
24/10/07 07:37:22 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
24/10/07 07:37:23 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
